## Batting df

In [2]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = r"C:\Users\mayur\Downloads\IPL 2024 data.xlsx"
ipl_data = pd.read_excel(file_path)

#dropping unneccesary column
ipl_data.drop(['other_player_dismissed','other_wicket_type'],axis=1,inplace=True)

#ipl_data['player_dismissed']= ipl_data['player_dismissed'].fillna(np.nan,inplace=True)
ipl_data['wicket_type']= ipl_data['wicket_type'].fillna(np.nan,inplace=True)

# Ensure 'noballs' and 'wides' columns are numeric
ipl_data['noballs'] = pd.to_numeric(ipl_data['noballs'], errors='coerce').fillna(0).astype(int)
ipl_data['wides'] = pd.to_numeric(ipl_data['wides'], errors='coerce').fillna(0).astype(int)

#Overall runs
ipl_data['total_runs'] = ipl_data['runs_off_bat'] + ipl_data['extras']

# Group data by batting team
grouped = ipl_data.groupby('batting_team')

# Initialize an empty DataFrame for the batting statistics
batting_stats = pd.DataFrame()

# Calculate the required statistics for each team
batting_stats['Team'] = grouped['batting_team'].first()
batting_stats['Matches'] = grouped['match_id'].nunique()

# Correct calculation for Innings: count the unique occurrences of batting_team and innings
batting_stats['Innings'] = ipl_data.groupby(['batting_team', 'match_id'])['innings'].nunique().groupby('batting_team').sum()

batting_stats['RunsScored'] = grouped['runs_off_bat'].sum()

# Calculate BallsFaced excluding no balls and wides
valid_balls = ipl_data[(ipl_data['noballs'] == 0) & (ipl_data['wides'] == 0)]
balls_faced = valid_balls.groupby('batting_team')['ball'].count()
batting_stats['BallsFaced'] = balls_faced

batting_stats['Outs'] = ipl_data[ipl_data['player_dismissed']!=np.nan].groupby('batting_team')['player_dismissed'].count()
batting_stats['Average'] = round(batting_stats['RunsScored'] / batting_stats['Outs'],2)
batting_stats['StrikeRate'] = round((batting_stats['RunsScored'] / batting_stats['BallsFaced']) * 100,2)

# Calculate the total number of fours and sixes
fours = ipl_data[ipl_data['runs_off_bat'] == 4].groupby('batting_team').size()
sixes = ipl_data[ipl_data['runs_off_bat'] == 6].groupby('batting_team').size()
batting_stats['4s'] = fours
batting_stats['6s'] = sixes

# Replace NaN with 0 for teams that have no fours or sixes
batting_stats['4s'].fillna(0, inplace=True)
batting_stats['6s'].fillna(0, inplace=True)

# Calculate the total number of dot balls
dot_balls = ipl_data[(ipl_data['runs_off_bat'] == 0) & (ipl_data['wides'] == 0)].groupby('batting_team').size()
batting_stats['DotBalls'] = dot_balls

# Replace NaN with 0 for teams that have no dot balls
batting_stats['DotBalls'].fillna(0, inplace=True)

# Calculate the highest team score in a single innings
team_scores = ipl_data.groupby(['batting_team', 'match_id'])['total_runs'].sum().reset_index()
highest_team_scores = team_scores.groupby('batting_team')['total_runs'].max().reset_index()
highest_team_scores.columns = ['batting_team', 'HighestScore']

# Merge the highest team scores with the batting_stats DataFrame
batting_stats = batting_stats.merge(highest_team_scores, on='batting_team', how='left')

batting_stats.drop(['batting_team'],axis=1,inplace=True)

# Runs of Boundaries
batting_stats['RunsInBoundaries']=(batting_stats['4s']*4)+(batting_stats['6s']*6)
#BoundaryRuns%
batting_stats['BoundaryRuns%']=round(batting_stats['RunsInBoundaries']*100/batting_stats['RunsScored'],2)

#Dots%
batting_stats['Dots%']=round(batting_stats['DotBalls']*100/batting_stats['BallsFaced'],2)

#balls/boundary
num_boundaries=batting_stats['4s']+batting_stats['6s']
batting_stats['balls/boundary']=round(batting_stats['BallsFaced']/num_boundaries,2)

#balls/dismissal
batting_stats['balls/dismissal']=round(batting_stats['BallsFaced']/batting_stats['Outs'],2)

#StrikeRotation%
batting_stats['StrikeRotation%']= 100-batting_stats['BoundaryRuns%']

# Display the batting_stats DataFrame
print(batting_stats)

                          Team  Matches  Innings  RunsScored  BallsFaced  \
0          Chennai Super Kings       14       14        2405        1648   
1               Delhi Capitals       14       14        2465        1576   
2               Gujarat Titans       12       12        1934        1405   
3        Kolkata Knight Riders       14       14        2481        1494   
4         Lucknow Super Giants       14       14        2350        1644   
5               Mumbai Indians       14       14        2422        1606   
6                 Punjab Kings       14       14        2372        1624   
7             Rajasthan Royals       15       15        2533        1747   
8  Royal Challengers Bengaluru       15       15        2789        1732   
9          Sunrisers Hyderabad       16       16        2906        1823   

   Outs  Average  StrikeRate   4s   6s  DotBalls  HighestScore  \
0    77    31.23      145.93  209  107       533           212   
1    94    26.22      156.41  2

In [3]:
batting_stats

,Team,Matches,Innings,RunsScored,BallsFaced,Outs,Average,StrikeRate,4s,6s,DotBalls,HighestScore,RunsInBoundaries,BoundaryRuns%,Dots%,balls/boundary,balls/dismissal,StrikeRotation%
0,Chennai Super Kings,14,14,2405,1648,77,31.23,145.93,209,107,533,212,1478,61.46,32.34,5.22,21.40,38.54
1,Delhi Capitals,14,14,2465,1576,94,26.22,156.41,229,135,574,257,1726,70.02,36.42,4.33,16.77,29.98
2,Gujarat Titans,12,12,1934,1405,79,24.48,137.65,183,68,481,231,1140,58.95,34.23,5.60,17.78,41.05
3,Kolkata Knight Riders,14,14,2481,1494,76,32.64,166.06,238,141,538,272,1798,72.47,36.01,3.94,19.66,27.53
4,Lucknow Super Giants,14,14,2350,1644,84,27.98,142.94,206,102,557,214,1436,61.11,33.88,5.34,19.57,38.89
5,Mumbai Indians,14,14,2422,1606,96,25.23,150.81,215,133,603,247,1658,68.46,37.55,4.61,16.73,31.54
6,Punjab Kings,14,14,2372,1624,92,25.78,146.06,213,120,595,262,1572,66.27,36.64,4.88,17.65,33.73
7,Rajasthan Royals,15,15,2533,1747,81,31.27,144.99,236,112,611,224,1616,63.80,34.97,5.02,21.57,36.20
8,Royal Challengers Bengaluru,15,15,2789,1732,99,28.17,161.03,229,165,571,262,1906,68.34,32.97,4.40,17.49,31.66
9,Sunrisers Hyderabad,16,16,2906,1823,105,27.68,159.41,217,178,603,287,1936,66.62,33.08,4.62,17.36,33.38


In [6]:
batting_stats.to_csv('batting_stats.csv',index=False)

## Bowling df

In [2]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = r"C:\Users\mayur\Downloads\IPL 2024 data.xlsx"
ipl_data = pd.read_excel(file_path)

#dropping unneccesary column
ipl_data.drop(['other_player_dismissed','other_wicket_type'],axis=1,inplace=True)

#ipl_data['player_dismissed']= ipl_data['player_dismissed'].fillna(np.nan,inplace=True)
#ipl_data['wicket_type']= ipl_data['wicket_type'].fillna(np.nan,inplace=True)

# Ensure relevant columns are numeric
ipl_data['noballs'] = pd.to_numeric(ipl_data['noballs'], errors='coerce').fillna(0).astype(int)
ipl_data['wides'] = pd.to_numeric(ipl_data['wides'], errors='coerce').fillna(0).astype(int)
ipl_data['runs_off_bat'] = pd.to_numeric(ipl_data['runs_off_bat'], errors='coerce').fillna(0).astype(int)
ipl_data['extras'] = pd.to_numeric(ipl_data['extras'], errors='coerce').fillna(0).astype(int)
ipl_data['player_dismissed'] = ipl_data['player_dismissed'].notnull().astype(int)
ipl_data['byes'] = pd.to_numeric(ipl_data['byes'], errors='coerce').fillna(0).astype(int)
ipl_data['legbyes'] = pd.to_numeric(ipl_data['legbyes'], errors='coerce').fillna(0).astype(int)

# Group data by bowling team
grouped = ipl_data.groupby('bowling_team')

# Initialize an empty DataFrame for the bowling statistics
bowling_stats = pd.DataFrame()

# Calculate the required statistics for each team
bowling_stats['Team'] = grouped['bowling_team'].first()
bowling_stats['Matches'] = grouped['match_id'].nunique()

# Correct calculation for Innings: count the unique occurrences of bowling_team and innings
bowling_stats['Innings'] = ipl_data.groupby(['bowling_team', 'match_id'])['innings'].nunique().groupby('bowling_team').sum()

# Calculate runs conceded
bowling_stats['RunsConceded'] = grouped['runs_off_bat'].sum() + grouped['extras'].sum()

# Calculate balls bowled excluding no balls and wides
valid_balls = ipl_data[(ipl_data['noballs'] == 0) & (ipl_data['wides'] == 0)]
balls_bowled = valid_balls.groupby('bowling_team')['ball'].count()
bowling_stats['BallsBowled'] = balls_bowled

# Calculate wickets taken
bowling_stats['WicketsTaken'] = grouped['player_dismissed'].sum()

# Calculate economy rate
bowling_stats['Economy'] = round((bowling_stats['RunsConceded'] / bowling_stats['BallsBowled']) * 6,2)

# Calculate bowling average and handle division by zero
bowling_stats['Average'] = round(bowling_stats['RunsConceded'] / bowling_stats['WicketsTaken'],2)
bowling_stats['Average'].replace([float('inf'), -float('inf')], float('nan'), inplace=True)

# Calculate strike rate and handle division by zero
bowling_stats['StrikeRate'] = round(bowling_stats['BallsBowled'] / bowling_stats['WicketsTaken'],2)
bowling_stats['StrikeRate'].replace([float('inf'), -float('inf')], float('nan'), inplace=True)

# Initialize columns for best bowling innings (BBI), 4W, 6W
bowling_stats['BBI'] = grouped.apply(lambda x: x.groupby(['match_id', 'innings'])['player_dismissed'].sum().max())
bowling_stats['10W'] = grouped.apply(lambda x: (x.groupby(['match_id', 'innings'])['player_dismissed'].sum() >= 10).sum())


# Calculate the total number of fours and sixes conceded
fours_conceded = ipl_data[ipl_data['runs_off_bat'] == 4].groupby('bowling_team').size()
sixes_conceded = ipl_data[ipl_data['runs_off_bat'] == 6].groupby('bowling_team').size()
bowling_stats['4s'] = fours_conceded
bowling_stats['6s'] = sixes_conceded

# Replace NaN with 0 for teams that have no fours or sixes conceded
bowling_stats['4s'].fillna(0, inplace=True)
bowling_stats['6s'].fillna(0, inplace=True)

# Calculate the total number of dot balls (including byes and legbyes)
dot_balls_conceded = ipl_data[(ipl_data['runs_off_bat'] == 0) & 
                              (ipl_data['wides'] == 0)].groupby('bowling_team').size()
bowling_stats['Dots'] = dot_balls_conceded

# Replace NaN with 0 for teams that have no dot balls conceded
bowling_stats['Dots'].fillna(0, inplace=True)

bowling_stats.drop(['Team'],axis=1,inplace=True)

# Runs of Boundaries
bowling_stats['RunsInBoundaries']=(bowling_stats['4s']*4)+(bowling_stats['6s']*6)
#BoundaryRuns%
bowling_stats['BoundaryRuns%']=round(bowling_stats['RunsInBoundaries']*100/bowling_stats['RunsConceded'],2)

#Dots%
bowling_stats['Dots%']=round(bowling_stats['Dots']*100/bowling_stats['BallsBowled'],2)

#balls/boundary
num_boundaries=bowling_stats['4s']+bowling_stats['6s']
bowling_stats['balls/boundary']=round(bowling_stats['BallsBowled']/num_boundaries,2)

#balls/dismissal
bowling_stats['balls/dismissal']=round(bowling_stats['BallsBowled']/bowling_stats['WicketsTaken'],2)

#StrikeRotation%
bowling_stats['StrikeRotation%']= 100-bowling_stats['BoundaryRuns%']

# Display the bowling_stats DataFrame
print(bowling_stats)


                             Matches  Innings  RunsConceded  BallsBowled  \
bowling_team                                                               
Chennai Super Kings               14       14          2415         1640   
Delhi Capitals                    14       14          2762         1640   
Gujarat Titans                    12       12          2101         1310   
Kolkata Knight Riders             14       14          2407         1576   
Lucknow Super Giants              14       14          2521         1566   
Mumbai Indians                    14       14          2660         1611   
Punjab Kings                      14       14          2612         1665   
Rajasthan Royals                  15       15          2657         1783   
Royal Challengers Bengaluru       15       15          2820         1709   
Sunrisers Hyderabad               16       16          3016         1799   

                             WicketsTaken  Economy  Average  StrikeRate  BBI  \
bowling

In [3]:
bowling_stats

,Matches,Innings,RunsConceded,BallsBowled,WicketsTaken,Economy,Average,StrikeRate,BBI,10W,4s,6s,Dots,RunsInBoundaries,BoundaryRuns%,Dots%,balls/boundary,balls/dismissal,StrikeRotation%
bowling_team,,,,,,,,,,,,,,,,,,,
Chennai Super Kings,14,14,2415,1640,79,8.84,30.57,20.76,10,1,198,103,582,1410,58.39,35.49,5.45,20.76,41.61
Delhi Capitals,14,14,2762,1640,99,10.10,27.90,16.57,10,1,232,144,560,1792,64.88,34.15,4.36,16.57,35.12
Gujarat Titans,12,12,2101,1310,71,9.62,29.59,18.45,10,1,174,109,452,1350,64.26,34.50,4.63,18.45,35.74
Kolkata Knight Riders,14,14,2407,1576,110,9.16,21.88,14.33,10,6,184,134,629,1540,63.98,39.91,4.96,14.33,36.02
Lucknow Super Giants,14,14,2521,1566,71,9.66,35.51,22.06,10,2,216,123,553,1602,63.55,35.31,4.62,22.06,36.45
Mumbai Indians,14,14,2660,1611,85,9.91,31.29,18.95,10,2,217,136,559,1684,63.31,34.70,4.56,18.95,36.69
Punjab Kings,14,14,2612,1665,101,9.41,25.86,16.49,9,0,230,114,570,1604,61.41,34.23,4.84,16.49,38.59
Rajasthan Royals,15,15,2657,1783,96,8.94,27.68,18.57,9,0,220,109,593,1534,57.73,33.26,5.42,18.57,42.27
Royal Challengers Bengaluru,15,15,2820,1709,88,9.90,32.05,19.42,10,3,239,143,585,1814,64.33,34.23,4.47,19.42,35.67


In [4]:
bowling_stats.to_csv('bowling_stats.csv')

## Extras df

In [12]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = r"C:\Users\mayur\Downloads\IPL 2024 data.xlsx"
ipl_data = pd.read_excel(file_path)


#dropping unneccesary column
ipl_data.drop(['other_player_dismissed','other_wicket_type'],axis=1,inplace=True)

ipl_data['player_dismissed']= ipl_data['player_dismissed'].fillna(np.nan,inplace=True)
ipl_data['wicket_type']= ipl_data['wicket_type'].fillna(np.nan,inplace=True)

# Ensure relevant columns are numeric
ipl_data['wides'] = pd.to_numeric(ipl_data['wides'], errors='coerce').fillna(0).astype(int)
ipl_data['legbyes'] = pd.to_numeric(ipl_data['legbyes'], errors='coerce').fillna(0).astype(int)
ipl_data['byes'] = pd.to_numeric(ipl_data['byes'], errors='coerce').fillna(0).astype(int)
ipl_data['noballs'] = pd.to_numeric(ipl_data['noballs'], errors='coerce').fillna(0).astype(int)
ipl_data['runs_off_bat'] = pd.to_numeric(ipl_data['runs_off_bat'], errors='coerce').fillna(0).astype(int)
ipl_data['extras'] = pd.to_numeric(ipl_data['extras'], errors='coerce').fillna(0).astype(int)

# Group data by bowling team
grouped_extras = ipl_data.groupby('bowling_team')

# Initialize an empty DataFrame for the extras statistics
extras_stats = pd.DataFrame()

# Calculate the required statistics for each team
extras_stats['Team'] = grouped_extras['bowling_team'].first()
extras_stats['Wides'] = grouped_extras['wides'].sum()
extras_stats['LegByes'] = grouped_extras['legbyes'].sum()
extras_stats['Byes'] = grouped_extras['byes'].sum()
extras_stats['NoBalls'] = grouped_extras['noballs'].sum()

# Calculate the number of each type of run (0s, 1s, 2s, 3s, 4s, 5s, 6s)
for run_value in range(7):
    extras_stats[f'{run_value}s'] = (ipl_data[ipl_data['runs_off_bat'] == run_value]
                                     .groupby('bowling_team').size().reindex(extras_stats['Team'], fill_value=0))

# Calculate the number of balls faced
extras_stats['BallsFaced'] = grouped_extras['ball'].count().reindex(extras_stats['Team'], fill_value=0)

# Calculate total runs conceded including extras
extras_stats['RunsOffBat'] = grouped_extras['runs_off_bat'].sum().reindex(extras_stats['Team'], fill_value=0)
extras_stats['TotalExtras'] = (grouped_extras['wides'].sum() + 
                               grouped_extras['legbyes'].sum() + 
                               grouped_extras['byes'].sum() + 
                               grouped_extras['noballs'].sum()).reindex(extras_stats['Team'], fill_value=0)
extras_stats['TotalRunsConceded'] = extras_stats['RunsOffBat'] + extras_stats['TotalExtras']

# Calculate total runs from individual counts to cross-check
extras_stats['CalculatedTotalRuns'] = (extras_stats['0s'] * 0 +
                                       extras_stats['1s'] * 1 +
                                       extras_stats['2s'] * 2 +
                                       extras_stats['3s'] * 3 +
                                       extras_stats['4s'] * 4 +
                                       extras_stats['5s'] * 5 +
                                       extras_stats['6s'] * 6 + 
                                       extras_stats['TotalExtras'])

# Fill any NaN values with 0
extras_stats.fillna(0, inplace=True)

# Display the extras_stats DataFrame to check for consistency
print(extras_stats)


                                                    Team  Wides  LegByes  \
bowling_team                                                               
Chennai Super Kings                  Chennai Super Kings     70       25   
Delhi Capitals                            Delhi Capitals    100       33   
Gujarat Titans                            Gujarat Titans     38       30   
Kolkata Knight Riders              Kolkata Knight Riders     71       26   
Lucknow Super Giants                Lucknow Super Giants     76       21   
Mumbai Indians                            Mumbai Indians     91       29   
Punjab Kings                                Punjab Kings     68       39   
Rajasthan Royals                        Rajasthan Royals     90       22   
Royal Challengers Bengaluru  Royal Challengers Bengaluru     87       27   
Sunrisers Hyderabad                  Sunrisers Hyderabad     75       22   

                             Byes  NoBalls   0s   1s   2s  3s   4s  5s   6s  \
bowling_

In [64]:
import pandas as pd

# Load the dataset
file_path = r"C:\Users\mayur\Downloads\IPL 2024 data.xlsx"
ipl_data = pd.read_excel(file_path)

# Ensure relevant columns are numeric
ipl_data['wides'] = pd.to_numeric(ipl_data['wides'], errors='coerce').fillna(0).astype(int)
ipl_data['legbyes'] = pd.to_numeric(ipl_data['legbyes'], errors='coerce').fillna(0).astype(int)
ipl_data['byes'] = pd.to_numeric(ipl_data['byes'], errors='coerce').fillna(0).astype(int)
ipl_data['noballs'] = pd.to_numeric(ipl_data['noballs'], errors='coerce').fillna(0).astype(int)
ipl_data['runs_off_bat'] = pd.to_numeric(ipl_data['runs_off_bat'], errors='coerce').fillna(0).astype(int)
ipl_data['extras'] = pd.to_numeric(ipl_data['extras'], errors='coerce').fillna(0).astype(int)

# Define a function to calculate the total runs conceded, including extras
def calculate_total_runs(row):
    return row['runs_off_bat'] + row['wides'] + row['legbyes'] + row['byes'] + row['noballs']

# Apply the function to calculate total runs for each delivery
ipl_data['total_runs'] = ipl_data.apply(calculate_total_runs, axis=1)

# Group data by bowling team
grouped_extras = ipl_data.groupby('bowling_team')

# Initialize an empty DataFrame for the extras statistics
extras_stats = pd.DataFrame()

# Calculate the required statistics for each team
extras_stats['Team'] = grouped_extras['bowling_team'].first()
extras_stats['Wides'] = grouped_extras['wides'].sum()
extras_stats['LegByes'] = grouped_extras['legbyes'].sum()
extras_stats['Byes'] = grouped_extras['byes'].sum()
extras_stats['NoBalls'] = grouped_extras['noballs'].sum()

# Calculate the number of each type of run (0s, 1s, 2s, 3s, 4s, 5s, 6s)
for run_value in range(7):
    extras_stats[f'{run_value}s'] = (ipl_data[ipl_data['total_runs'] == run_value]
                                     .groupby('bowling_team').size().reindex(extras_stats['Team'], fill_value=0))

# Calculate total runs conceded including extras
extras_stats['RunsOffBat'] = grouped_extras['runs_off_bat'].sum().reindex(extras_stats['Team'], fill_value=0)
extras_stats['TotalExtras'] = (grouped_extras['wides'].sum() + 
                               grouped_extras['legbyes'].sum() + 
                               grouped_extras['byes'].sum() + 
                               grouped_extras['noballs'].sum()).reindex(extras_stats['Team'], fill_value=0)
extras_stats['TotalRunsConceded'] = grouped_extras['total_runs'].sum().reindex(extras_stats['Team'], fill_value=0)

# Calculate total runs from individual counts to cross-check
extras_stats['CalculatedTotalRuns'] = (extras_stats['0s'] * 0 +
                                       extras_stats['1s'] * 1 +
                                       extras_stats['2s'] * 2 +
                                       extras_stats['3s'] * 3 +
                                       extras_stats['4s'] * 4 +
                                       extras_stats['5s'] * 5 +
                                       extras_stats['6s'] * 6)

# Fill any NaN values with 0
extras_stats.fillna(0, inplace=True)

# Display the extras_stats DataFrame to check for consistency
print(extras_stats)


                                                    Team  Wides  LegByes  \
bowling_team                                                               
Chennai Super Kings                  Chennai Super Kings     77       26   
Delhi Capitals                            Delhi Capitals    100       33   
Gujarat Titans                            Gujarat Titans     38       30   
Kolkata Knight Riders              Kolkata Knight Riders     82       28   
Lucknow Super Giants                Lucknow Super Giants     86       21   
Mumbai Indians                            Mumbai Indians    102       29   
Punjab Kings                                Punjab Kings     78       45   
Rajasthan Royals                        Rajasthan Royals     99       25   
Royal Challengers Bengaluru  Royal Challengers Bengaluru    100       29   
Sunrisers Hyderabad                  Sunrisers Hyderabad     89       39   

                             Byes  NoBalls   0s   1s   2s  3s   4s  5s   6s  \
bowling_

In [15]:
extras_stats

,Team,Wides,LegByes,Byes,NoBalls,0s,1s,2s,3s,4s,5s,6s,RunsOffBat,TotalExtras,TotalRunsConceded,CalculatedTotalRuns
bowling_team,,,,,,,,,,,,,,,,
Chennai Super Kings,Chennai Super Kings,70,25,9,12,514,698,114,0,182,4,86,2081,116,2197,2190
Delhi Capitals,Delhi Capitals,100,33,9,9,524,720,109,3,235,2,143,2611,151,2762,2755
Gujarat Titans,Gujarat Titans,38,30,4,2,429,539,91,2,177,2,107,2027,74,2101,2087
Kolkata Knight Riders,Kolkata Knight Riders,71,26,10,6,501,527,87,1,164,5,125,2022,113,2135,2135
Lucknow Super Giants,Lucknow Super Giants,76,21,2,10,480,609,102,8,199,4,112,2216,109,2325,2325
Mumbai Indians,Mumbai Indians,91,29,7,12,485,654,103,3,203,6,119,2307,139,2446,2425
Punjab Kings,Punjab Kings,68,39,5,3,507,677,104,7,219,3,100,2282,115,2397,2397
Rajasthan Royals,Rajasthan Royals,90,22,6,3,493,714,109,1,198,6,91,2189,121,2310,2303
Royal Challengers Bengaluru,Royal Challengers Bengaluru,87,27,11,5,489,628,94,4,208,3,130,2325,130,2455,2455


## Phase wise batting df

In [3]:
def categorize_overs(overs):
    if overs < 6.0:
        return 'Power play'
    elif overs < 15.0:
        return 'Middle overs'
    else:
        return 'Death'

In [4]:
ipl_data['over_category'] = ipl_data["ball"].apply(categorize_overs)

In [5]:
#Overall runs
ipl_data['total_runs'] = ipl_data['runs_off_bat'] + ipl_data['extras']

In [6]:
matches = ipl_data.groupby(['batting_team','over_category'])['match_id'].nunique().reset_index()
matches.rename(columns = {'match_id':'Matches'}, inplace = True)
matches

,batting_team,over_category,Matches
0,Chennai Super Kings,Death,14
1,Chennai Super Kings,Middle overs,14
2,Chennai Super Kings,Power play,14
3,Delhi Capitals,Death,13
4,Delhi Capitals,Middle overs,14
5,Delhi Capitals,Power play,14
6,Gujarat Titans,Death,12
7,Gujarat Titans,Middle overs,12
8,Gujarat Titans,Power play,12
9,Kolkata Knight Riders,Death,12


In [7]:
dot_balls = ipl_data[(ipl_data['runs_off_bat'] == 0) & (ipl_data['wides'] == 0)].groupby(['batting_team','over_category'])['ball'].size().reset_index()
dot_balls.rename(columns = {'ball':'dotballs'}, inplace = True)
dot_balls

,batting_team,over_category,dotballs
0,Chennai Super Kings,Death,111
1,Chennai Super Kings,Middle overs,221
2,Chennai Super Kings,Power play,201
3,Delhi Capitals,Death,106
4,Delhi Capitals,Middle overs,236
5,Delhi Capitals,Power play,232
6,Gujarat Titans,Death,101
7,Gujarat Titans,Middle overs,188
8,Gujarat Titans,Power play,192
9,Kolkata Knight Riders,Death,83


In [8]:
runs=ipl_data.groupby(['batting_team','over_category'])['runs_off_bat'].sum().reset_index()
runs.rename(columns = {'runs_off_bat':'Runs'}, inplace = True)
runs

,batting_team,over_category,Runs
0,Chennai Super Kings,Death,700
1,Chennai Super Kings,Middle overs,1007
2,Chennai Super Kings,Power play,698
3,Delhi Capitals,Death,611
4,Delhi Capitals,Middle overs,1005
5,Delhi Capitals,Power play,849
6,Gujarat Titans,Death,544
7,Gujarat Titans,Middle overs,864
8,Gujarat Titans,Power play,526
9,Kolkata Knight Riders,Death,518


In [9]:
wickets=ipl_data[ipl_data['player_dismissed']!='np.nan'].groupby(['batting_team','over_category'])['player_dismissed'].sum().reset_index()
wickets.rename(columns = {'player_dismissed':'wickets'}, inplace = True)
wickets

,batting_team,over_category,wickets
0,Chennai Super Kings,Death,24
1,Chennai Super Kings,Middle overs,34
2,Chennai Super Kings,Power play,19
3,Delhi Capitals,Death,29
4,Delhi Capitals,Middle overs,37
5,Delhi Capitals,Power play,28
6,Gujarat Titans,Death,30
7,Gujarat Titans,Middle overs,33
8,Gujarat Titans,Power play,16
9,Kolkata Knight Riders,Death,27


In [10]:
fours=ipl_data[ipl_data['runs_off_bat']==4].groupby(['batting_team','over_category'])['runs_off_bat'].count().reset_index()
fours.rename(columns = {'runs_off_bat':'4s'}, inplace = True)
fours

,batting_team,over_category,4s
0,Chennai Super Kings,Death,50
1,Chennai Super Kings,Middle overs,68
2,Chennai Super Kings,Power play,91
3,Delhi Capitals,Death,45
4,Delhi Capitals,Middle overs,75
5,Delhi Capitals,Power play,109
6,Gujarat Titans,Death,55
7,Gujarat Titans,Middle overs,69
8,Gujarat Titans,Power play,59
9,Kolkata Knight Riders,Death,39


In [11]:
sixes=ipl_data[ipl_data['runs_off_bat']==6].groupby(['batting_team','over_category'])['runs_off_bat'].count().reset_index()
sixes.rename(columns = {'runs_off_bat':'6s'}, inplace = True)
sixes

,batting_team,over_category,6s
0,Chennai Super Kings,Death,48
1,Chennai Super Kings,Middle overs,41
2,Chennai Super Kings,Power play,18
3,Delhi Capitals,Death,39
4,Delhi Capitals,Middle overs,50
5,Delhi Capitals,Power play,46
6,Gujarat Titans,Death,24
7,Gujarat Titans,Middle overs,28
8,Gujarat Titans,Power play,16
9,Kolkata Knight Riders,Death,38


In [12]:
Balls=ipl_data[(ipl_data['noballs']==0) &(ipl_data['wides']==0)].groupby(['batting_team','over_category'])['extras'].count().reset_index()
Balls.rename(columns = {'extras':'Balls'}, inplace = True)
Balls

,batting_team,over_category,Balls
0,Chennai Super Kings,Death,388
1,Chennai Super Kings,Middle overs,756
2,Chennai Super Kings,Power play,504
3,Delhi Capitals,Death,353
4,Delhi Capitals,Middle overs,719
5,Delhi Capitals,Power play,504
6,Gujarat Titans,Death,325
7,Gujarat Titans,Middle overs,648
8,Gujarat Titans,Power play,432
9,Kolkata Knight Riders,Death,269


In [13]:
phase_batting_df=pd.merge(matches,runs, on=['batting_team','over_category'], how='outer')
phase_batting_df

,batting_team,over_category,Matches,Runs
0,Chennai Super Kings,Death,14,700
1,Chennai Super Kings,Middle overs,14,1007
2,Chennai Super Kings,Power play,14,698
3,Delhi Capitals,Death,13,611
4,Delhi Capitals,Middle overs,14,1005
5,Delhi Capitals,Power play,14,849
6,Gujarat Titans,Death,12,544
7,Gujarat Titans,Middle overs,12,864
8,Gujarat Titans,Power play,12,526
9,Kolkata Knight Riders,Death,12,518


In [14]:
phase_batting_df=pd.merge(phase_batting_df,Balls,on=['batting_team','over_category'], how='outer')
phase_batting_df=pd.merge(phase_batting_df,wickets,on=['batting_team','over_category'], how='outer')
phase_batting_df=pd.merge(phase_batting_df,fours,on=['batting_team','over_category'], how='outer')
phase_batting_df=pd.merge(phase_batting_df,sixes,on=['batting_team','over_category'], how='outer')
phase_batting_df=pd.merge(phase_batting_df,dot_balls,on=['batting_team','over_category'], how='outer')
phase_batting_df

,batting_team,over_category,Matches,Runs,Balls,wickets,4s,6s,dotballs
0,Chennai Super Kings,Death,14,700,388,24,50,48,111
1,Chennai Super Kings,Middle overs,14,1007,756,34,68,41,221
2,Chennai Super Kings,Power play,14,698,504,19,91,18,201
3,Delhi Capitals,Death,13,611,353,29,45,39,106
4,Delhi Capitals,Middle overs,14,1005,719,37,75,50,236
5,Delhi Capitals,Power play,14,849,504,28,109,46,232
6,Gujarat Titans,Death,12,544,325,30,55,24,101
7,Gujarat Titans,Middle overs,12,864,648,33,69,28,188
8,Gujarat Titans,Power play,12,526,432,16,59,16,192
9,Kolkata Knight Riders,Death,12,518,269,27,39,38,83


In [15]:
phase_batting_df.insert(6,'Average',round(phase_batting_df['Runs']/phase_batting_df['wickets'],2))
phase_batting_df

,batting_team,over_category,Matches,Runs,Balls,wickets,Average,4s,6s,dotballs
0,Chennai Super Kings,Death,14,700,388,24,29.17,50,48,111
1,Chennai Super Kings,Middle overs,14,1007,756,34,29.62,68,41,221
2,Chennai Super Kings,Power play,14,698,504,19,36.74,91,18,201
3,Delhi Capitals,Death,13,611,353,29,21.07,45,39,106
4,Delhi Capitals,Middle overs,14,1005,719,37,27.16,75,50,236
5,Delhi Capitals,Power play,14,849,504,28,30.32,109,46,232
6,Gujarat Titans,Death,12,544,325,30,18.13,55,24,101
7,Gujarat Titans,Middle overs,12,864,648,33,26.18,69,28,188
8,Gujarat Titans,Power play,12,526,432,16,32.88,59,16,192
9,Kolkata Knight Riders,Death,12,518,269,27,19.19,39,38,83


In [16]:
phase_batting_df.insert(7,'Strike rate',round(phase_batting_df['Runs']*100/phase_batting_df['Balls'],2))
phase_batting_df

,batting_team,over_category,Matches,Runs,Balls,wickets,Average,Strike rate,4s,6s,dotballs
0,Chennai Super Kings,Death,14,700,388,24,29.17,180.41,50,48,111
1,Chennai Super Kings,Middle overs,14,1007,756,34,29.62,133.20,68,41,221
2,Chennai Super Kings,Power play,14,698,504,19,36.74,138.49,91,18,201
3,Delhi Capitals,Death,13,611,353,29,21.07,173.09,45,39,106
4,Delhi Capitals,Middle overs,14,1005,719,37,27.16,139.78,75,50,236
5,Delhi Capitals,Power play,14,849,504,28,30.32,168.45,109,46,232
6,Gujarat Titans,Death,12,544,325,30,18.13,167.38,55,24,101
7,Gujarat Titans,Middle overs,12,864,648,33,26.18,133.33,69,28,188
8,Gujarat Titans,Power play,12,526,432,16,32.88,121.76,59,16,192
9,Kolkata Knight Riders,Death,12,518,269,27,19.19,192.57,39,38,83


In [17]:
team_runs=ipl_data.groupby(['match_id','batting_team','over_category'])['total_runs'].sum().reset_index()
high_score = team_runs.loc[team_runs.groupby(['batting_team','over_category'])['total_runs'].idxmax()]
score_df=high_score[['batting_team','over_category','total_runs']]
score_df.rename(columns={'total_runs':'High score'},inplace=True)
score_df

C:\Users\mayur\AppData\Local\Temp\ipykernel_20200\2970777328.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df.rename(columns={'total_runs':'High score'},inplace=True)


,batting_team,over_category,High score
227,Chennai Super Kings,Death,75
169,Chennai Super Kings,Middle overs,101
38,Chennai Super Kings,Power play,69
233,Delhi Capitals,Death,97
91,Delhi Capitals,Middle overs,108
253,Delhi Capitals,Power play,92
138,Gujarat Titans,Death,75
349,Gujarat Titans,Middle overs,132
238,Gujarat Titans,Power play,67
12,Kolkata Knight Riders,Death,85


In [18]:
phase_batting_df=pd.merge(phase_batting_df,score_df,on=['batting_team','over_category'])
phase_batting_df

,batting_team,over_category,Matches,Runs,Balls,wickets,Average,Strike rate,4s,6s,dotballs,High score
0,Chennai Super Kings,Death,14,700,388,24,29.17,180.41,50,48,111,75
1,Chennai Super Kings,Middle overs,14,1007,756,34,29.62,133.20,68,41,221,101
2,Chennai Super Kings,Power play,14,698,504,19,36.74,138.49,91,18,201,69
3,Delhi Capitals,Death,13,611,353,29,21.07,173.09,45,39,106,97
4,Delhi Capitals,Middle overs,14,1005,719,37,27.16,139.78,75,50,236,108
5,Delhi Capitals,Power play,14,849,504,28,30.32,168.45,109,46,232,92
6,Gujarat Titans,Death,12,544,325,30,18.13,167.38,55,24,101,75
7,Gujarat Titans,Middle overs,12,864,648,33,26.18,133.33,69,28,188,132
8,Gujarat Titans,Power play,12,526,432,16,32.88,121.76,59,16,192,67
9,Kolkata Knight Riders,Death,12,518,269,27,19.19,192.57,39,38,83,85


In [19]:
# Runs of Boundaries
phase_batting_df['RunsInBoundaries']=(phase_batting_df['4s']*4)+(phase_batting_df['6s']*6)
#BoundaryRuns%
phase_batting_df['BoundaryRuns%']=round(phase_batting_df['RunsInBoundaries']*100/phase_batting_df['Runs'],2)

#Dots%
phase_batting_df['Dots%']=round(phase_batting_df['dotballs']*100/phase_batting_df['Balls'],2)

#balls/boundary
num_boundaries=phase_batting_df['4s']+phase_batting_df['6s']
phase_batting_df['balls/boundary']=round(phase_batting_df['Balls']/num_boundaries,2)

#balls/dismissal
phase_batting_df['balls/dismissal']=round(phase_batting_df['Balls']/phase_batting_df['wickets'],2)

#StrikeRotation%
phase_batting_df['StrikeRotation%']= 100-phase_batting_df['BoundaryRuns%']

phase_batting_df

,batting_team,over_category,Matches,Runs,Balls,wickets,Average,Strike rate,4s,6s,dotballs,High score,RunsInBoundaries,BoundaryRuns%,Dots%,balls/boundary,balls/dismissal,StrikeRotation%
0,Chennai Super Kings,Death,14,700,388,24,29.17,180.41,50,48,111,75,488,69.71,28.61,3.96,16.17,30.29
1,Chennai Super Kings,Middle overs,14,1007,756,34,29.62,133.20,68,41,221,101,518,51.44,29.23,6.94,22.24,48.56
2,Chennai Super Kings,Power play,14,698,504,19,36.74,138.49,91,18,201,69,472,67.62,39.88,4.62,26.53,32.38
3,Delhi Capitals,Death,13,611,353,29,21.07,173.09,45,39,106,97,414,67.76,30.03,4.20,12.17,32.24
4,Delhi Capitals,Middle overs,14,1005,719,37,27.16,139.78,75,50,236,108,600,59.70,32.82,5.75,19.43,40.30
5,Delhi Capitals,Power play,14,849,504,28,30.32,168.45,109,46,232,92,712,83.86,46.03,3.25,18.00,16.14
6,Gujarat Titans,Death,12,544,325,30,18.13,167.38,55,24,101,75,364,66.91,31.08,4.11,10.83,33.09
7,Gujarat Titans,Middle overs,12,864,648,33,26.18,133.33,69,28,188,132,444,51.39,29.01,6.68,19.64,48.61
8,Gujarat Titans,Power play,12,526,432,16,32.88,121.76,59,16,192,67,332,63.12,44.44,5.76,27.00,36.88
9,Kolkata Knight Riders,Death,12,518,269,27,19.19,192.57,39,38,83,85,384,74.13,30.86,3.49,9.96,25.87


In [20]:
print(phase_batting_df)

                   batting_team over_category  Matches  Runs  Balls  wickets  \
0           Chennai Super Kings         Death       14   700    388       24   
1           Chennai Super Kings  Middle overs       14  1007    756       34   
2           Chennai Super Kings    Power play       14   698    504       19   
3                Delhi Capitals         Death       13   611    353       29   
4                Delhi Capitals  Middle overs       14  1005    719       37   
5                Delhi Capitals    Power play       14   849    504       28   
6                Gujarat Titans         Death       12   544    325       30   
7                Gujarat Titans  Middle overs       12   864    648       33   
8                Gujarat Titans    Power play       12   526    432       16   
9         Kolkata Knight Riders         Death       12   518    269       27   
10        Kolkata Knight Riders  Middle overs       14  1114    721       28   
11        Kolkata Knight Riders    Power

In [21]:
phase_batting_df.to_csv('phasewise_batting_df.csv',index=False)

## Phase wise Bowling df

In [29]:
Matches=ipl_data.groupby(['bowling_team','over_category'])['match_id'].nunique().reset_index()
Matches.rename(columns = {'match_id':'Matches'}, inplace = True)
Matches

,bowling_team,over_category,Matches
0,Chennai Super Kings,Death,14
1,Chennai Super Kings,Middle overs,14
2,Chennai Super Kings,Power play,14
3,Delhi Capitals,Death,14
4,Delhi Capitals,Middle overs,14
5,Delhi Capitals,Power play,14
6,Gujarat Titans,Death,10
7,Gujarat Titans,Middle overs,12
8,Gujarat Titans,Power play,12
9,Kolkata Knight Riders,Death,14


In [30]:
Dots=ipl_data[(ipl_data['wides']==0) & (ipl_data['runs_off_bat']==0)].groupby(['bowling_team','over_category'])['ball'].count().reset_index()
Dots.rename(columns = {'ball':'dotballs'}, inplace = True)
Dots

,bowling_team,over_category,dotballs
0,Chennai Super Kings,Death,128
1,Chennai Super Kings,Middle overs,230
2,Chennai Super Kings,Power play,224
3,Delhi Capitals,Death,111
4,Delhi Capitals,Middle overs,230
5,Delhi Capitals,Power play,219
6,Gujarat Titans,Death,71
7,Gujarat Titans,Middle overs,187
8,Gujarat Titans,Power play,194
9,Kolkata Knight Riders,Death,126


In [31]:
Runs_conceeded=ipl_data.groupby(['bowling_team','over_category'])['total_runs'].sum().reset_index()
Runs_conceeded.rename(columns = {'total_runs':'Runs_conceeded'}, inplace = True)
Runs_conceeded

,bowling_team,over_category,Runs_conceeded
0,Chennai Super Kings,Death,671
1,Chennai Super Kings,Middle overs,1007
2,Chennai Super Kings,Power play,737
3,Delhi Capitals,Death,788
4,Delhi Capitals,Middle overs,1078
5,Delhi Capitals,Power play,896
6,Gujarat Titans,Death,524
7,Gujarat Titans,Middle overs,891
8,Gujarat Titans,Power play,686
9,Kolkata Knight Riders,Death,526


In [32]:
Balls_bowled=ipl_data[(ipl_data['noballs']==0) &(ipl_data['wides']==0)].groupby(['bowling_team','over_category'])['extras'].count().reset_index()
Balls_bowled.rename(columns = {'extras':'balls_bowled'}, inplace = True)
Balls_bowled

,bowling_team,over_category,balls_bowled
0,Chennai Super Kings,Death,380
1,Chennai Super Kings,Middle overs,756
2,Chennai Super Kings,Power play,504
3,Delhi Capitals,Death,380
4,Delhi Capitals,Middle overs,756
5,Delhi Capitals,Power play,504
6,Gujarat Titans,Death,275
7,Gujarat Titans,Middle overs,603
8,Gujarat Titans,Power play,432
9,Kolkata Knight Riders,Death,316


In [33]:
Wickets_taken=ipl_data[ipl_data['player_dismissed']!='np.nan'].groupby(['bowling_team','over_category'])['player_dismissed'].sum().reset_index()
Wickets_taken

,bowling_team,over_category,player_dismissed
0,Chennai Super Kings,Death,29
1,Chennai Super Kings,Middle overs,35
2,Chennai Super Kings,Power play,15
3,Delhi Capitals,Death,33
4,Delhi Capitals,Middle overs,41
5,Delhi Capitals,Power play,25
6,Gujarat Titans,Death,24
7,Gujarat Titans,Middle overs,24
8,Gujarat Titans,Power play,23
9,Kolkata Knight Riders,Death,31


In [34]:
Fours_conceeded=ipl_data[ipl_data['runs_off_bat']==4].groupby(['bowling_team','over_category'])['runs_off_bat'].count().reset_index()
Fours_conceeded.rename(columns = {'runs_off_bat':'fours_conceeded'}, inplace = True)
Fours_conceeded

,bowling_team,over_category,fours_conceeded
0,Chennai Super Kings,Death,51
1,Chennai Super Kings,Middle overs,64
2,Chennai Super Kings,Power play,83
3,Delhi Capitals,Death,48
4,Delhi Capitals,Middle overs,72
5,Delhi Capitals,Power play,112
6,Gujarat Titans,Death,28
7,Gujarat Titans,Middle overs,68
8,Gujarat Titans,Power play,78
9,Kolkata Knight Riders,Death,21


In [35]:
Sixes_conceeded=ipl_data[ipl_data['runs_off_bat']==6].groupby(['bowling_team','over_category'])['runs_off_bat'].count().reset_index()
Sixes_conceeded.rename(columns = {'runs_off_bat':'sixes_conceeded'}, inplace = True)
Sixes_conceeded

,bowling_team,over_category,sixes_conceeded
0,Chennai Super Kings,Death,34
1,Chennai Super Kings,Middle overs,37
2,Chennai Super Kings,Power play,32
3,Delhi Capitals,Death,57
4,Delhi Capitals,Middle overs,48
5,Delhi Capitals,Power play,39
6,Gujarat Titans,Death,38
7,Gujarat Titans,Middle overs,39
8,Gujarat Titans,Power play,32
9,Kolkata Knight Riders,Death,45


In [36]:
bowling_df = ipl_data[['bowling_team', 'match_id', 'innings', 'over_category', 'wicket_type']].copy()

# Filter out rows without a wicket
bowling_df = bowling_df[bowling_df['wicket_type'].notnull()]

# Group by team, phase, match, and innings to count wickets
wickets_df = bowling_df.groupby(['bowling_team', 'over_category', 'match_id', 'innings']).size().reset_index(name='Most_wickets')

# Find the maximum wickets taken by each team in each phase
max_wickets = wickets_df.groupby(['bowling_team', 'over_category'])['Most_wickets'].max().reset_index()
max_wickets

,bowling_team,over_category,Most_wickets
0,Chennai Super Kings,Death,5
1,Chennai Super Kings,Middle overs,6
2,Chennai Super Kings,Power play,3
3,Delhi Capitals,Death,5
4,Delhi Capitals,Middle overs,5
5,Delhi Capitals,Power play,4
6,Gujarat Titans,Death,6
7,Gujarat Titans,Middle overs,5
8,Gujarat Titans,Power play,4
9,Kolkata Knight Riders,Death,4


In [37]:
phase_bowling_df=pd.merge(Matches,Runs_conceeded, on=['bowling_team','over_category'], how='outer')
phase_bowling_df

,bowling_team,over_category,Matches,Runs_conceeded
0,Chennai Super Kings,Death,14,671
1,Chennai Super Kings,Middle overs,14,1007
2,Chennai Super Kings,Power play,14,737
3,Delhi Capitals,Death,14,788
4,Delhi Capitals,Middle overs,14,1078
5,Delhi Capitals,Power play,14,896
6,Gujarat Titans,Death,10,524
7,Gujarat Titans,Middle overs,12,891
8,Gujarat Titans,Power play,12,686
9,Kolkata Knight Riders,Death,14,526


In [38]:
phase_bowling_df=pd.merge(phase_bowling_df,Balls_bowled,on=['bowling_team','over_category'], how='outer')
phase_bowling_df=pd.merge(phase_bowling_df,Wickets_taken,on=['bowling_team','over_category'], how='outer')
phase_bowling_df=pd.merge(phase_bowling_df,Dots,on=['bowling_team','over_category'], how='outer')
phase_bowling_df=pd.merge(phase_bowling_df,Fours_conceeded,on=['bowling_team','over_category'], how='outer')
phase_bowling_df=pd.merge(phase_bowling_df,Sixes_conceeded,on=['bowling_team','over_category'], how='outer')
phase_bowling_df=pd.merge(phase_bowling_df,max_wickets,on=['bowling_team','over_category'], how='outer')
phase_bowling_df

,bowling_team,over_category,Matches,Runs_conceeded,balls_bowled,player_dismissed,dotballs,fours_conceeded,sixes_conceeded,Most_wickets
0,Chennai Super Kings,Death,14,671,380,29,128,51,34,5
1,Chennai Super Kings,Middle overs,14,1007,756,35,230,64,37,6
2,Chennai Super Kings,Power play,14,737,504,15,224,83,32,3
3,Delhi Capitals,Death,14,788,380,33,111,48,57,5
4,Delhi Capitals,Middle overs,14,1078,756,41,230,72,48,5
5,Delhi Capitals,Power play,14,896,504,25,219,112,39,4
6,Gujarat Titans,Death,10,524,275,24,71,28,38,6
7,Gujarat Titans,Middle overs,12,891,603,24,187,68,39,5
8,Gujarat Titans,Power play,12,686,432,23,194,78,32,4
9,Kolkata Knight Riders,Death,14,526,316,31,126,21,45,4


In [39]:
phase_bowling_df.insert(7,'Economy', round((phase_bowling_df['Runs_conceeded'] / phase_bowling_df['balls_bowled']) * 6,2))
phase_bowling_df

,bowling_team,over_category,Matches,Runs_conceeded,balls_bowled,player_dismissed,dotballs,Economy,fours_conceeded,sixes_conceeded,Most_wickets
0,Chennai Super Kings,Death,14,671,380,29,128,10.59,51,34,5
1,Chennai Super Kings,Middle overs,14,1007,756,35,230,7.99,64,37,6
2,Chennai Super Kings,Power play,14,737,504,15,224,8.77,83,32,3
3,Delhi Capitals,Death,14,788,380,33,111,12.44,48,57,5
4,Delhi Capitals,Middle overs,14,1078,756,41,230,8.56,72,48,5
5,Delhi Capitals,Power play,14,896,504,25,219,10.67,112,39,4
6,Gujarat Titans,Death,10,524,275,24,71,11.43,28,38,6
7,Gujarat Titans,Middle overs,12,891,603,24,187,8.87,68,39,5
8,Gujarat Titans,Power play,12,686,432,23,194,9.53,78,32,4
9,Kolkata Knight Riders,Death,14,526,316,31,126,9.99,21,45,4


In [40]:
phase_bowling_df.insert(8,'Average', round(phase_bowling_df['Runs_conceeded'] / phase_bowling_df['player_dismissed'],2))
phase_bowling_df

,bowling_team,over_category,Matches,Runs_conceeded,balls_bowled,player_dismissed,dotballs,Economy,Average,fours_conceeded,sixes_conceeded,Most_wickets
0,Chennai Super Kings,Death,14,671,380,29,128,10.59,23.14,51,34,5
1,Chennai Super Kings,Middle overs,14,1007,756,35,230,7.99,28.77,64,37,6
2,Chennai Super Kings,Power play,14,737,504,15,224,8.77,49.13,83,32,3
3,Delhi Capitals,Death,14,788,380,33,111,12.44,23.88,48,57,5
4,Delhi Capitals,Middle overs,14,1078,756,41,230,8.56,26.29,72,48,5
5,Delhi Capitals,Power play,14,896,504,25,219,10.67,35.84,112,39,4
6,Gujarat Titans,Death,10,524,275,24,71,11.43,21.83,28,38,6
7,Gujarat Titans,Middle overs,12,891,603,24,187,8.87,37.12,68,39,5
8,Gujarat Titans,Power play,12,686,432,23,194,9.53,29.83,78,32,4
9,Kolkata Knight Riders,Death,14,526,316,31,126,9.99,16.97,21,45,4


In [41]:
phase_bowling_df.insert(9,'StrikeRate', round(phase_bowling_df['balls_bowled'] / phase_bowling_df['player_dismissed'],2))
phase_bowling_df

,bowling_team,over_category,Matches,Runs_conceeded,balls_bowled,player_dismissed,dotballs,Economy,Average,StrikeRate,fours_conceeded,sixes_conceeded,Most_wickets
0,Chennai Super Kings,Death,14,671,380,29,128,10.59,23.14,13.10,51,34,5
1,Chennai Super Kings,Middle overs,14,1007,756,35,230,7.99,28.77,21.60,64,37,6
2,Chennai Super Kings,Power play,14,737,504,15,224,8.77,49.13,33.60,83,32,3
3,Delhi Capitals,Death,14,788,380,33,111,12.44,23.88,11.52,48,57,5
4,Delhi Capitals,Middle overs,14,1078,756,41,230,8.56,26.29,18.44,72,48,5
5,Delhi Capitals,Power play,14,896,504,25,219,10.67,35.84,20.16,112,39,4
6,Gujarat Titans,Death,10,524,275,24,71,11.43,21.83,11.46,28,38,6
7,Gujarat Titans,Middle overs,12,891,603,24,187,8.87,37.12,25.12,68,39,5
8,Gujarat Titans,Power play,12,686,432,23,194,9.53,29.83,18.78,78,32,4
9,Kolkata Knight Riders,Death,14,526,316,31,126,9.99,16.97,10.19,21,45,4


In [42]:
# Runs of Boundaries
phase_bowling_df['RunsInBoundaries']=(phase_bowling_df['fours_conceeded']*4)+(phase_bowling_df['sixes_conceeded']*6)
#BoundaryRuns%
phase_bowling_df['BoundaryRuns%']=round(phase_bowling_df['RunsInBoundaries']*100/phase_bowling_df['Runs_conceeded'],2)

#Dots%
phase_bowling_df['Dots%']=round(phase_bowling_df['dotballs']*100/phase_bowling_df['balls_bowled'],2)

#balls/boundary
num_boundaries=phase_bowling_df['fours_conceeded']+phase_bowling_df['sixes_conceeded']
phase_bowling_df['balls/boundary']=round(phase_bowling_df['balls_bowled']/num_boundaries,2)

#balls/dismissal
phase_bowling_df['balls/dismissal']=round(phase_bowling_df['balls_bowled']/phase_bowling_df['player_dismissed'],2)

#StrikeRotation%
phase_bowling_df['StrikeRotation%']= 100-phase_bowling_df['BoundaryRuns%']

phase_bowling_df

,bowling_team,over_category,Matches,Runs_conceeded,balls_bowled,player_dismissed,dotballs,Economy,Average,StrikeRate,fours_conceeded,sixes_conceeded,Most_wickets,RunsInBoundaries,BoundaryRuns%,Dots%,balls/boundary,balls/dismissal,StrikeRotation%
0,Chennai Super Kings,Death,14,671,380,29,128,10.59,23.14,13.10,51,34,5,408,60.80,33.68,4.47,13.10,39.20
1,Chennai Super Kings,Middle overs,14,1007,756,35,230,7.99,28.77,21.60,64,37,6,478,47.47,30.42,7.49,21.60,52.53
2,Chennai Super Kings,Power play,14,737,504,15,224,8.77,49.13,33.60,83,32,3,524,71.10,44.44,4.38,33.60,28.90
3,Delhi Capitals,Death,14,788,380,33,111,12.44,23.88,11.52,48,57,5,534,67.77,29.21,3.62,11.52,32.23
4,Delhi Capitals,Middle overs,14,1078,756,41,230,8.56,26.29,18.44,72,48,5,576,53.43,30.42,6.30,18.44,46.57
5,Delhi Capitals,Power play,14,896,504,25,219,10.67,35.84,20.16,112,39,4,682,76.12,43.45,3.34,20.16,23.88
6,Gujarat Titans,Death,10,524,275,24,71,11.43,21.83,11.46,28,38,6,340,64.89,25.82,4.17,11.46,35.11
7,Gujarat Titans,Middle overs,12,891,603,24,187,8.87,37.12,25.12,68,39,5,506,56.79,31.01,5.64,25.12,43.21
8,Gujarat Titans,Power play,12,686,432,23,194,9.53,29.83,18.78,78,32,4,504,73.47,44.91,3.93,18.78,26.53
9,Kolkata Knight Riders,Death,14,526,316,31,126,9.99,16.97,10.19,21,45,4,354,67.30,39.87,4.79,10.19,32.70


In [150]:
print(phase_bowling_df)

                   bowling_team over_category  Matches  Runs_conceeded  \
0           Chennai Super Kings         Death       14             671   
1           Chennai Super Kings  Middle overs       14            1007   
2           Chennai Super Kings    Power play       14             737   
3                Delhi Capitals         Death       14             788   
4                Delhi Capitals  Middle overs       14            1078   
5                Delhi Capitals    Power play       14             896   
6                Gujarat Titans         Death       10             524   
7                Gujarat Titans  Middle overs       12             891   
8                Gujarat Titans    Power play       12             686   
9         Kolkata Knight Riders         Death       14             526   
10        Kolkata Knight Riders  Middle overs       14            1045   
11        Kolkata Knight Riders    Power play       14             836   
12         Lucknow Super Giants       

In [43]:
phase_bowling_df.to_csv('phasewise_bowling_df.csv',index=False)